In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# %load ../../loaders/imports.py
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import time
import pdb

# Add the uoicorr directory to the path
sys.path.append('../../../uoicorr_run')

# Add the root directory of this repository
sys.path.append('../..')

from utils import gen_covariance, gen_beta2, gen_data, get_cov_list, sparsify_beta
from utils import selection_accuracy
from sklearn.linear_model import LassoLars, lasso_path, LinearRegression

In [5]:
import pickle

In [3]:
# Grab the data files
sys.path.append('/media/akumar/Data/nse/data')

In [4]:
# Load the og_cov_params and and the newly generated spread. Then, expand the set by about 40 to make sure 
# we have an even tiling of all parameters across all sparsities

In [7]:
with open('og_cov_metrics.dat', 'rb') as f:
    og_rho = pickle.load(f)
    og_eta = pickle.load(f)
    og_eta2 = pickle.load(f)

In [23]:
with open('/media/akumar/Data/nse/data/cov_ensemble.dat', 'rb') as f:
    rho = pickle.load(f)
    eta = pickle.load(f)
    eta2 = pickle.load(f)
    norm_diff = pickle.load(f)

In [ ]:
# Algorithm:

# For each metric (rho, eta, eta2), and each sparsity, if applicable, iteratively go through the list of 
# alternatives and add the one that increases the variance of the distribution of values the most. Continue
# this procedure until the intersection of all additions equals 80. This is the final cov ensemble.

In [11]:
rho.shape

(80, 10, 20)

In [27]:
og_eta2.shape

(80, 14)

In [19]:
# rho - no sparsity dependence
rho = rho.ravel()


# calculate the variance of the og_rho distribution after adding each 
# element of rho to it. And then take the 

rho_inclusion_variance = np.zeros(rho.size)

for i, r in enumerate(rho):
    
    # add r to og_rho
    augmented_rho = np.append(og_rho, r)
    rho_inclusion_variance[i] = np.var(augmented_rho)
    

In [22]:
import copy

In [24]:
# eta: Average over the last axis as this is repetitive data draws. Ravel along the first three axes
# Then, we will have a unique ordering for each sparsity. 
eta_ = copy.deepcopy(eta)
eta_ = np.mean(eta_, axis = -1)
eta_ = np.reshape(eta_, (-1, 14))

eta_inclusion_variance = np.zeros(eta_.shape)

for i1 in range(eta_.shape[0]):
    for i2 in range(eta_.shape[1]):
        
        augmented_eta = np.append(og_eta[:, i2], eta_[i1, i2])
        eta_inclusion_variance[i1, i2] = 1/np.var(augmented_eta)


In [28]:
# eta2 : Same thing but we have no need to average over the last index
eta2_ = copy.deepcopy(eta2)
eta2_ = np.reshape(eta2_, (-1, 14))
eta2_inclusion_variance = np.zeros(eta2_.shape)

for i1 in range(eta2_.shape[0]):
    for i2 in range(eta2_.shape[1]):
        
        augmented_eta2 = np.append(og_eta2[:, i2], eta2_[i1,i2])
        eta2_inclusion_variance[i1, i2] = 1/np.var(augmented_eta2)

In [33]:
rho_sort_order = np.argsort(rho_inclusion_variance)[::-1]

In [39]:
eta_sort_order = np.argsort(eta_inclusion_variance, axis = 0)
eta2_sort_order = np.argsort(eta2_inclusion_variance, axis = 0)

In [40]:
# Take the intersection. Start at 80 elements and sequentially step up until the intersection across all 
# elements hits 80. If this does not work reasoanbly, then we can take just between rho and eta2

In [41]:
rho_sort_order.shape

(16000,)

In [52]:
intersection_set = set([])

idx = 80

while len(intersection_set) < 80 and idx < 16000:
    
    rho_set = set(rho_sort_order[:idx])
    eta_set_order = [set(eta_sort_order[:idx, i]) for i in range(eta_sort_order.shape[1])]
    eta2_set_order = [set(eta2_sort_order[:idx, i]) for i in range(eta2_sort_order.shape[1])]    
    eta_set_order.extend(eta2_set_order)
    pdb.set_trace()    
    intersection = rho_set.intersection(eta_set_order)
    intersection_set.union(intersection)
    idx += 1

> <ipython-input-52-801318c29037>(13)<module>()
-> intersection = rho_set.intersection(eta_set_order)
(Pdb) len(eta_set_order)
28
(Pdb) len(eta_set_order[0])
79
(Pdb) quit()


BdbQuit: 